# nbd

> Extra command line utilities for nbdev

In [ ]:
#| default_exp nbd

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from fastcore.all import *
import itertools as it
import os, time
from ghapi.all import *

import configparser
from pathlib import Path

from oztools.core import *
from oztools.gh import *

import subprocess

import json
from fastcore.net import HTTP422UnprocessableEntityError

from glob import glob
import re, yaml, shutil
import importlib.resources as res

import asyncio
from asyncinotify import Inotify, Event, Mask
from pathlib import Path
from typing import Generator, AsyncGenerator
from datetime import datetime

In [ ]:
path = Path("../..")
Path.BASE_PATH = path

In [ ]:
#| export
def make_things_pretty():
    # Fix not being able to click on "source" link in docs
    with open("./nbs/styles.css", 'a') as f: f.write("\nh3 {\n  width: fit-content;\n}")
    # Add dark theme and make bright theme compatible with in in colorscheme
    theme = {'light': 'united', 'dark': 'superhero'}
    with open("./nbs/_quarto.yml", 'r') as f: data = yaml.safe_load(f)
    data['format']['html']['theme'] = theme
    data['website']['favicon'] = 'favicon.png'
    with open("./nbs/_quarto.yml", 'w') as f: f.write(yaml.dump(data))
    # Copy favicon
    with res.as_file(res.files("oztools")/"data/favicon.png") as f: shutil.copy(f, "./nbs")

In [ ]:
#| export

def nbd_new_fn(name:str, description:str, license:str="Apache-2.0", private:bool=False):
    "Create a new nbdev project and setup github repo for it"
    gh_repo, local_repo = gh_new_repo_fn(name, description, license, private)
    # this one makes github run CI twice (not nice)
    #setup_pages_branch(local_repo, gh_repo.name)
    os.chdir(gh_repo.name)
    subprocess.run(["nbdev_new"]) # TODO: use .__wrapped__ property to extract original function
    subprocess.run(["nbdev_install_hooks"])
    make_things_pretty()
    subprocess.run(["nbdev_prepare"])
    subprocess.run(["nbdev_clean"])
    # Hopefully using pip + some sleep would be enough for github to be ready
    # to enable pages branch
    commit_and_push(local_repo, "Initial commit")
    subprocess.run(["pip", "install", "-e", ".[dev]"])
    print("Waiting for the github to finish build before enabling pages")
    print("Feel free to start working on the project")
    # TODO: maybe check github api if build is ready
    while True:
        time.sleep(20)
        try:
            setup_pages_branch_location(local_repo, gh_repo.name)
            break
        except HTTP422UnprocessableEntityError:
            pass

In [ ]:
#| export
@call_parse
def nbd_new(name:str, description:str, license:str="Apache-2.0", private:bool=False):
    "Create a new nbdev project and setup github repo for it"
    nbd_new_fn(name, description, license, private)

In [ ]:
#| export

def new_notebook_template(name, description):
    template = {
        'cells': [
            {'cell_type': 'markdown', 'metadata': {},
             'source': [
                    f'# {name}\n',
                    '\n',
                    f'> {description}'
                ]},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [f'#| default_exp {name}']},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [
                 '#| hide\n',
                 'from nbdev.showdoc import *'
             ]},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [
                 '#| export\n',
                 'from fastcore.all import *'
             ]},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [
                 '#| hide\n',
                 'import nbdev; nbdev.nbdev_e'+'xport()' # nbdev_e**ort is a forbidden word
             ]},
      ], 'metadata': { 'kernelspec': { 'display_name': 'python3', 'language': 'python', 'name': 'python3' } }, 'nbformat': 4, 'nbformat_minor': 4
    }
    return template

FIXME: Cell below results in an error for some reason

In [ ]:
template = new_notebook_template("foo", "Makes foo using bar")
print('\n---\n\n'.join(L(template['cells']).attrgot('source').map(''.join)))

# foo

> Makes foo using bar
---

#| default_exp foo
---

#| hide
from nbdev.showdoc import *
---

#| export
from fastcore.all import *
---

#| hide
import nbdev; nbdev.nbdev_export()


In [ ]:
m = re.match(r'(\d+).*\.ipynb', '99ab_asd.ipynb')
m.group(1)

'99'

In [ ]:
#| export
def zero_pad(num):
    num = str(num)
    return num if len(num) > 1 else f"0{num}"

In [ ]:
test_eq(zero_pad(9), '09')
test_eq(zero_pad(32), '32')

In [ ]:
#| export
@call_parse
def nbd_add(name:str, description:str,
            at:Optional[int] = None # If specified, insert new notebook at a specific position
           ):
    "Add new notebook to the project"

    prev_nbs = L(glob("[0-9]*?*.ipynb"))
    prev_ids = prev_nbs.map(lambda x: int(re.match(r'(\d+).*\.ipynb', x).group(1)))
    prev_id = max(prev_ids)

    new_id = prev_id + 1
    new_id = zero_pad(new_id)
    template = new_notebook_template(name, description)

    with open(f"{new_id}_{name}.ipynb", 'w') as f:
        json.dump(template, f)

## nbd_watch

Adapted from https://github.com/ProCern/asyncinotify/blob/master/examples/recursivewatch.py

In [ ]:
#| export
def get_directories_recursive(path: Path):
    if path.is_dir():
        yield path
        for child in path.iterdir(): yield from get_directories_recursive(child)

In [ ]:
list(get_directories_recursive(Path('.')))

[Path('.'),
 Path('.ipynb_checkpoints'),
 Path('02_nbd'),
 Path('02_nbd/favicons'),
 Path('02_nbd/favicons/arlantr'),
 Path('02_nbd/favicons/food-ocal'),
 Path('02_nbd/favicons/food-ocal/vegetable'),
 Path('02_nbd/favicons/food-ocal/treat'),
 Path('02_nbd/favicons/food-ocal/pasta'),
 Path('02_nbd/favicons/food-ocal/fruit'),
 Path('02_nbd/favicons/food-ocal/drink'),
 Path('02_nbd/favicons/food-ocal/dairy'),
 Path('02_nbd/favicons/food-ocal/meat')]

In [ ]:
import asyncinotify

In [ ]:
asyncinotify.InitFlags??

Init signature: asyncinotify.InitFlags(*values)
Source:        
class InitFlags(IntFlag):
    '''Init flags for use with the :class:`Inotify` constructor.

    You shouldn't have a reason to use this, as :attr:`CLOEXEC` will be desired
    because there's no reason for exec'd children to inherit inotify handles
    here, and :attr:`NONBLOCK` shouldn't even make a difference due to the
    handle always being watched with select, unless you are using synchronous mode.
    '''

    __slots__ = ()

    #: Set the close-on-exec (FD_CLOEXEC) flag on the new file descriptor.  See
    #: the description of the O_CLOEXEC flag in  open(2)  for  reasons why this
    #: may be useful.
    CLOEXEC = os.O_CLOEXEC

    #: Set the O_NONBLOCK file status flag on the open file description (see
    #: open(2)) referred to by the new file descriptor.  Using this flag saves
    #: extra calls to fcntl(2) to achieve the same result.
    NONBLOCK = os.O_NONBLOCK
File:           ~/python/venv/lib/python3.12/

In [ ]:
Inotify??

Init signature:
Inotify(
    flags: asyncinotify.InitFlags = <InitFlags.CLOEXEC|NONBLOCK: 526336>,
    cache_size: int = 10,
    sync_timeout: Optional[float] = None,
) -> None
Source:        
class Inotify:
    '''Core Inotify class.

    Fetches events in bulk, if possible, and stores them internally.

    Use :meth:`get` to get a single event.  This class operates as an async
    generator, and may be asynchronously iterated, and will return events
    forever.

    :param int cache_size: The max number of full-size events to cache.  The
        actual number may be higher, because most events will not be
        full-sized.

    :param sync_timeout: If this is not None, then sync_get will wait on an
        epoll call for that long, and return None on a timeout.  Normal
        iteration will also exit on a timeout.
    '''

    __slots__ = ('_fd', '_watches', '_events', '_epoll', '_sync_timeout', '_cache_size', '__weakref__')

    def __init__(self,
                 flags: InitFla

In [ ]:
#| export
async def watch_recursive(path: Path, sync_timeout=None) -> AsyncGenerator[Event, None]:
    mask = Mask.CREATE | Mask.MODIFY
    with Inotify(sync_timeout=sync_timeout) as inotify:
        for directory in get_directories_recursive(path):
            inotify.add_watch(directory, mask)
        async for event in inotify:
            if Mask.CREATE in event.mask and event.path is not None and event.path.is_dir():
                for directory in get_directories_recursive(event.path):
                    #print(f'EVENT: watching {directory}')
                    inotify.add_watch(directory, mask)

            if event.mask & mask:
                yield event

In [ ]:
#| export
async def debounce(events: AsyncGenerator[Event, None], # event generator
                   debounce_interval = 0.5 # in seconds
                  ) -> AsyncGenerator[Event, None]:
    combined_event = []
    last_event_time = None

    async for event in events:
        current_time = time.time()

        if last_event_time is None or (current_time - last_event_time) <= debounce_interval:
            combined_event.append(event)
        else:
            if combined_event:
                yield combined_event
                combined_event = [event]
        last_event_time = current_time

    print("This will never run for inotify")
    if combined_event:
        yield combined_event

In [ ]:
async def event_generator():
    await asyncio.sleep(0.25)
    yield 1
    await asyncio.sleep(0.25)
    yield 2
    await asyncio.sleep(0.25)
    yield 3
    await asyncio.sleep(0.25)
    yield 4
    await asyncio.sleep(0.25)
    yield 5
    await asyncio.sleep(0.25)
    yield 6
    await asyncio.sleep(0.5)
    yield 7

In [ ]:
aloop = asyncio.get_event_loop()

In [ ]:
async def run_loop(event_gen):
    async for e in event_gen:
        print(e)

In [ ]:
aloop.create_task(run_loop(event_generator()))

<Task pending name='Task-5' coro=<run_loop() running at /tmp/ipykernel_899807/2799408329.py:1>>

1
2
3
4
5
6
7


In [ ]:
aloop.create_task(run_loop(debounce(event_generator())))

<Task pending name='Task-6' coro=<run_loop() running at /tmp/ipykernel_899807/2799408329.py:1>>

[1, 2, 3, 4, 5, 6]
This will never run for inotify
[7]


In [ ]:
#| export
async def inotify_debounce(path, debounce_interval=1):
    print("inotify_debounce")
    combined_event = []
    last_event_time = None

    while True:
        # NOTE: this is not busy waiting because watch_recursive acts as a sleep function
        # This process only wakes up either every debounce interval or when directory events happen
        async for event in watch_recursive(path, debounce_interval):
            print(event)
            current_time = time.time()
    
            if last_event_time is None or (current_time - last_event_time) <= debounce_interval:
                combined_event.append(event)
            else:
                if combined_event:
                    yield combined_event
                    combined_event = [event]
            last_event_time = current_time
    
        print("This will never run for inotify")
        if combined_event:
            yield combined_event

In [ ]:
#| export
async def nbd_watch_async():
    #async for el in debounce(watch_recursive(Path('nbs')), 1):
    async for el in inotify_debounce(Path('nbs')):
        print(f"[{datetime.now()}] {el[0].path}")
        subprocess.run(["nbdev_export"])

In [ ]:
#| export
def nbd_watch():
    "Watch `nbs` folder and automatically run `nbdev_export` on file change"
    asyncio.run(nbd_watch_async())

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()